In [ ]:
!pip install gspread
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
import pandas as pd
import numpy as np
import json
import os
import gspread
import google.auth
from google.cloud import bigquery
from google.colab import userdata
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.colab import drive
from datetime import datetime
import plotly.express as px
import seaborn as sns

In [ ]:
service_account_key_path = '/content/sa_key.json'

# Write JSON string to a file
with open(service_account_key_path, 'w') as file:
    file.write(userdata.get('sa_key'))

# Set the environment variable for the Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key_path

# Authenticate using the service account JSON key
credentials = service_account.Credentials.from_service_account_file(
    service_account_key_path, scopes=["https://www.googleapis.com/auth/drive"]
)

# Build the Drive API service
drive_service = build('drive', 'v3', credentials=credentials)

# Ingest Pizza Price


In [ ]:
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets.readonly"]

# Authenticate with the service account
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/sa_key.json', scope)

# Authorize the gspread client
client = gspread.authorize(creds)

sheet = client.open('Master Pizza').sheet1

data = sheet.get_all_records()

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data)


In [ ]:
df.head()

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pizza_id       96 non-null     object 
 1   pizza_type_id  96 non-null     object 
 2   size           96 non-null     object 
 3   price          96 non-null     float64
dtypes: float64(1), object(3)
memory usage: 3.1+ KB


In [ ]:
df.shape[0]

96

In [ ]:
df.isna().sum()

,0
pizza_id,0
pizza_type_id,0
size,0
price,0


In [ ]:
df = df.astype({
    'pizza_id': 'string',
    'pizza_type_id': 'string',
    'size': 'string',
    'price': 'float',
})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pizza_id       96 non-null     string 
 1   pizza_type_id  96 non-null     string 
 2   size           96 non-null     string 
 3   price          96 non-null     float64
dtypes: float64(1), string(3)
memory usage: 3.1 KB


In [ ]:
client = bigquery.Client(project='tough-variety-418711')

query = f"""
CREATE OR REPLACE TABLE `tough-variety-418711.projects.dim_pizza_price` (
  pizza_id STRING,
  pizza_type_id STRING,
  size STRING,
  price FLOAT64,
);

"""

client.query(query)


QueryJob<project=tough-variety-418711, location=US, id=8a985372-3759-46b3-965c-85f8561d9925>

In [ ]:
from google.cloud import bigquery
import pandas_gbq

project_id = 'tough-variety-418711'
table_id = 'dim_pizza_price'
dataset_id = 'projects'

# Full table path: 'project.dataset.table'
table_full_id = f'{project_id}.{dataset_id}.{table_id}'

# Write DataFrame to BigQuery table
df.to_gbq(destination_table=table_full_id,
                   project_id=project_id,
                   if_exists='append')  # 'replace' if you want to overwrite

<ipython-input-60-1f84e60d5ef2>:12: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=table_full_id,
100%|██████████| 1/1 [00:00<00:00, 7133.17it/s]


# Ingest Pizza Details


In [ ]:
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets.readonly"]

# Authenticate with the service account
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/sa_key.json', scope)

# Authorize the gspread client
client = gspread.authorize(creds)

sheet = client.open('Pizza Details').sheet1

data = sheet.get_all_records()

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data)


In [ ]:
df.head()

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   pizza_type_id  32 non-null     object
 1   name           32 non-null     object
 2   category       32 non-null     object
 3   ingredients    32 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


In [ ]:
df.shape[0]

32

In [ ]:
df.isna().sum()

,0
pizza_type_id,0
name,0
category,0
ingredients,0


In [ ]:
df = df.astype({
    'pizza_type_id': 'string',
    'name': 'string',
    'category': 'string',
    'ingredients': 'string',
})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   pizza_type_id  32 non-null     string
 1   name           32 non-null     string
 2   category       32 non-null     string
 3   ingredients    32 non-null     string
dtypes: string(4)
memory usage: 1.1 KB


In [ ]:
client = bigquery.Client(project='tough-variety-418711')

query = f"""
CREATE OR REPLACE TABLE `tough-variety-418711.projects.dim_pizza_detail` (
  pizza_type_id STRING,
  name STRING,
  category STRING,
  ingredients STRING,
);

"""

client.query(query)


QueryJob<project=tough-variety-418711, location=US, id=9471712c-9f04-4167-9a73-fc5c658b96fc>

In [ ]:
from google.cloud import bigquery
import pandas_gbq

project_id = 'tough-variety-418711'
table_id = 'dim_pizza_detail'
dataset_id = 'projects'

# Full table path: 'project.dataset.table'
table_full_id = f'{project_id}.{dataset_id}.{table_id}'

# Write DataFrame to BigQuery table
df.to_gbq(destination_table=table_full_id,
                   project_id=project_id,
                   if_exists='append')  # 'replace' if you want to overwrite

<ipython-input-69-b8ed2d549e9b>:12: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=table_full_id,
100%|██████████| 1/1 [00:00<00:00, 5275.85it/s]


# Ingest Pizza Order


In [ ]:
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets.readonly"]

# Authenticate with the service account
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/sa_key.json', scope)

# Authorize the gspread client
client = gspread.authorize(creds)

sheet = client.open('Pizza Order').sheet1

data = sheet.get_all_records()

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data)


In [ ]:
df.head()

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21350 entries, 0 to 21349
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   order_id  21350 non-null  int64 
 1   date      21350 non-null  object
 2   time      21350 non-null  object
dtypes: int64(1), object(2)
memory usage: 500.5+ KB


In [ ]:
df.shape[0]

21350

In [ ]:
df.isna().sum()

,0
order_id,0
date,0
time,0


In [ ]:
df = df.astype({
    'order_id': 'int',
})

# Convert date and time columns to proper formats first, then to string
df['date'] = pd.to_datetime(df['date']).dt.date.astype('string')
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21350 entries, 0 to 21349
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   order_id  21350 non-null  int64 
 1   date      21350 non-null  string
 2   time      21350 non-null  object
dtypes: int64(1), object(1), string(1)
memory usage: 500.5+ KB


In [ ]:
df.head()

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30


In [ ]:
client = bigquery.Client(project='tough-variety-418711')

query = f"""
CREATE OR REPLACE TABLE `tough-variety-418711.projects.dim_pizza_order` (
  order_id INT64,
  date DATE,
  time TIME
);

"""

client.query(query)


QueryJob<project=tough-variety-418711, location=US, id=d6545307-bbc0-417c-8814-b4ac18150604>

In [ ]:
from google.cloud import bigquery
import pandas_gbq

project_id = 'tough-variety-418711'
table_id = 'dim_pizza_order'
dataset_id = 'projects'

# Full table path: 'project.dataset.table'
table_full_id = f'{project_id}.{dataset_id}.{table_id}'

# Write DataFrame to BigQuery table
df.to_gbq(destination_table=table_full_id,
                   project_id=project_id,
                   if_exists='append')  # 'replace' if you want to overwrite

<ipython-input-79-e9010e64b2c5>:12: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=table_full_id,
100%|██████████| 1/1 [00:00<00:00, 9776.93it/s]


# Ingest Pizza Order Detail



In [ ]:
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets.readonly"]

# Authenticate with the service account
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/sa_key.json', scope)

# Authorize the gspread client
client = gspread.authorize(creds)

sheet = client.open('Pizza Order Detail').sheet1

data = sheet.get_all_records()

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data)


In [ ]:
df.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_details_id  48620 non-null  int64 
 1   order_id          48620 non-null  int64 
 2   pizza_id          48620 non-null  object
 3   quantity          48620 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [ ]:
df.shape[0]

48620

In [ ]:
df.isna().sum()

,0
order_details_id,0
order_id,0
pizza_id,0
quantity,0


In [ ]:
df = df.astype({
    'order_details_id': 'int',
    'order_id': 'int',
    'pizza_id': 'string',
    'quantity': 'int',
})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_details_id  48620 non-null  int64 
 1   order_id          48620 non-null  int64 
 2   pizza_id          48620 non-null  string
 3   quantity          48620 non-null  int64 
dtypes: int64(3), string(1)
memory usage: 1.5 MB


In [ ]:
client = bigquery.Client(project='tough-variety-418711')

query = f"""
CREATE OR REPLACE TABLE `tough-variety-418711.projects.dwd_pizza_order_detail` (
  order_details_id INT64,
  order_id INT64,
  pizza_id STRING,
  quantity INT64,
);

"""

client.query(query)


QueryJob<project=tough-variety-418711, location=US, id=47c2546b-3e0a-45c2-bced-1be573ee3d6a>

In [ ]:
from google.cloud import bigquery
import pandas_gbq

project_id = 'tough-variety-418711'
table_id = 'dwd_pizza_order_detail'
dataset_id = 'projects'

# Full table path: 'project.dataset.table'
table_full_id = f'{project_id}.{dataset_id}.{table_id}'

# Write DataFrame to BigQuery table
df.to_gbq(destination_table=table_full_id,
                   project_id=project_id,
                   if_exists='append')  # 'replace' if you want to overwrite

<ipython-input-88-0811e766fc75>:12: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=table_full_id,
100%|██████████| 1/1 [00:00<00:00, 2211.02it/s]
